## Word2Vec

In [ ]:
! pip install gensim

In [1]:
from gensim.models.word2vec import Word2Vec
import pandas as pd
import gensim
from konlpy.tag import Okt

In [14]:
#  Nsmc 데이터 로드
train_df = pd.read_csv('./data/ratings_train.csv')
train_df.dropna(inplace=True)

In [11]:
# 데이터 확인
train_df[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
# 불용어 정의
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '을', '를', '는', '으로']

In [9]:
# 명사 추출 및 불용어 제거하여 데이터 전처리하기
# 1. 문서 하나씩 읽어오기 (50개만)
# 2. 문서에 okt 형태소 분석 적용 (morphs함수, stem=True 변경)
# 3. 불용어 제거
# 4. tokenized_words 리스트(문서 별 형태소 추출): [[words, words, words], [words, words, words]]
# 5. sentence_nouns 리스트(모든 명사 리스트): [[noun, noun, noun, noun], [noun, noun, noun, noun]]
okt = Okt()

tokenized_words, sentence_nouns = [] , []

for sentence in train_df['document'][:50]:
    # 형태소 분석 토큰화
    tokenized_word = okt.morphs(sentence, stem=True) # 토큰화 및 정규화 -> 어간을 추출
    stopwords_removed_sentence = [word for word in tokenized_word if not word in stopwords]
    tokenized_words.append(stopwords_removed_sentence)

    # 명사만 추출
    nouns = okt.nouns(sentence)
    sentence_nouns.append(nouns) 

In [11]:
# 리뷰의 최대 길이
word_len = [len(l) for l in tokenized_words]
nouns_len = [len(l) for l in sentence_nouns]

print(f"가장 긴 리뷰 길이 : {max(word_len)}, 가장 짧은 리뷰 길이 : {min(word_len)}")
print(f"가장 긴 리뷰(명사) 길이 : {max(nouns_len)}, 가장 짧은 리뷰(명사) 길이 : {min(nouns_len)}")

# 가장 긴 리뷰 길이 : 95, 가장 짧은 리뷰 길이 : 1
# 가장 긴 리뷰(명사) 길이 : 66, 가장 짧은 리뷰(명사) 길이 : 0

가장 긴 단어 길이 : 60, 가장 짧은 단어 길이 : 1
가장 긴 명사 길이 : 29, 가장 짧은 명사 길이 : 1


## Word2Vec 훈련

vector_size = 워드 벡터의 특징 값, 즉 임베딩 된 벡터의 차원  
window = 컨텐스트 윈도우 크기  
min_count = 단어 최소 빈도 수 제한(빈도가 적은 단어들을 학습하지 않도록 하는 기준)  
workers = 학습을 위한 프로세스 수 (workers로 설정을 하거나 혹은 multiprocessing으로 병렬처리)  
sg = 0 (CBOW) , 1 (Skip-gram)  

In [13]:
# word2vec 모델 훈련하기 -> no run
model_cbow = Word2Vec(tokenized_words, vector_size=100, window=5, min_count=5, workers=4, sg =0)
model_skipgram = Word2Vec(tokenized_words, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [14]:
# 완성된 임베딩 매트릭스의 크기 확인(단어수, 차원수) -> no run
print(model_cbow.wv.vectors.shape)
print(model_skipgram.wv.vectors.shape)

(15204, 100)
(15204, 100)


모델 저장 방법

1. 재훈련 없이 모델만 사용
2. 재훈련을 할 수 있도록 임베딩에 대한 기본 정보를 함께 저장

In [15]:
# 모델 저장 방법 1 : 재훈련을 하지 않는 경우 
## Ram에 로드하지 않고도 디스크나 네트워크에서 데이터를 즉시 읽어 반복할 수 있음
## 모델 inference 에서 활용하면 좋음 
## -> no run

model_cbow.wv.save_word2vec_format('./data/kor_w2v_cbow')
model_skipgram.wv.save_word2vec_format('./data/kor_w2v_skipgram')

In [12]:
# 모델 저장 방법1의 결과를 다시 로드해서 사용

from gensim.models import KeyedVectors

# 모델 불러오기
model_cbow_kv = KeyedVectors.load_word2vec_format('./data/kor_w2v_cbow')
model_skipgram_kv = KeyedVectors.load_word2vec_format('./data/kor_w2v_skipgram')

In [13]:
# 특정 단어를 중심으로 유사한 단어 확인하기

print(model_cbow_kv.most_similar("한석규", topn=5))
print(model_skipgram_kv.most_similar("한석규", topn=5))

[('설경구', 0.9148017168045044), ('차승원', 0.8984052538871765), ('황정민', 0.8980780839920044), ('안성기', 0.8957299590110779), ('김혜수', 0.8939363360404968)]
[('안성기', 0.8880666494369507), ('설경구', 0.8817006349563599), ('최민식', 0.8728572130203247), ('최민수', 0.8718010187149048), ('신들리다', 0.8633845448493958)]


In [19]:
# 모델 저장 방법2 : 재훈련 가능한 모델 
# -> no run

model_cbow.save('./data/kor_w2v_cbow.model')
model_skipgram.save('./data/kor_w2v_skipgram.model')

In [14]:
# 모델 불러오기
## 모델에 vocab 정보 등이 함께 저장된 결과를 로드함

model_cbow_model = Word2Vec.load('./data/kor_w2v_cbow.model')
model_skipgram_model = Word2Vec.load('./data/kor_w2v_skipgram.model')

In [15]:
# 특정 단어를 중심으로 유사한 단어 확인하기

print(model_cbow_model.wv.most_similar('추천'))
print(model_skipgram_model.wv.most_similar('추천'))

[('꼭', 0.6887606978416443), ('강추', 0.6834990382194519), ('후회', 0.6812189221382141), ('강력', 0.602091372013092), ('수고', 0.5819746255874634), ('감상', 0.5753079652786255), ('적극', 0.575278639793396), ('추하다', 0.5751488208770752), ('권하다', 0.5735456943511963), ('소장', 0.570164680480957)]
[('강력', 0.7862173914909363), ('강추', 0.7671416997909546), ('적극', 0.7580629587173462), ('권하다', 0.727484941482544), ('추하다', 0.7162095308303833), ('해드리다', 0.7140164375305176), ('보삼', 0.6767366528511047), ('불면증', 0.6703829169273376), ('꼭', 0.6552332043647766), ('예매', 0.647720217704773)]


In [16]:
# 두 단어 간의 유사도 파악하기
model_skipgram_model.wv.similarity('추천', '강력')

0.78621733

In [17]:
model_skipgram_model.wv.most_similar("영화")

[('멜로영화', 0.7437077164649963),
 ('청춘영화', 0.7317690253257751),
 ('만화영화', 0.7294827103614807),
 ('공포영화', 0.7282743453979492),
 ('괴수영화', 0.724732518196106),
 ('액션영화', 0.7229880094528198),
 ('판타지영화', 0.7190592288970947),
 ('애니매이션', 0.7132899761199951),
 ('공포물', 0.7101417779922485),
 ('로맨스영화', 0.7087157964706421)]

In [18]:
model_skipgram_model.wv.most_similar("인터스텔라")

[('데스노트', 0.8372506499290466),
 ('초속', 0.8157038688659668),
 ('4-5', 0.8087527751922607),
 ('4~5', 0.8024197220802307),
 ('편임', 0.8012981414794922),
 ('갓파더', 0.7976078391075134),
 ('은별', 0.7930920124053955),
 ('1~4', 0.7910516262054443),
 ('딥블루씨', 0.7888678908348083),
 ('어메이징', 0.7884843349456787)]

In [20]:
# 사전에 없는 경우 확인
model_skipgram_model.wv.most_similar("파묘")

KeyError: "Key '파묘' not present in vocabulary"

In [22]:
# 여러 단어와 유사한 embedding 가져오기 -> negative: 안 가져옴
model_skipgram_model.wv.most_similar(positive=['여자', '감독'], negative=['남자'])

[('김기덕', 0.626354992389679),
 ('역량', 0.6012963652610779),
 ('제작자', 0.5873257517814636),
 ('장진', 0.5859320163726807),
 ('박찬욱', 0.5739708542823792),
 ('자질', 0.5664553642272949),
 ('작가', 0.5588862895965576),
 ('오우삼', 0.5537692308425903),
 ('봉준호', 0.5470575094223022),
 ('임권택', 0.5425761342048645)]

In [23]:
model_skipgram_model.wv.most_similar(positive=['남자', '감독'], negative=['여자'])

[('김기덕', 0.6410437226295471),
 ('장진', 0.6115475296974182),
 ('봉준호', 0.5934121608734131),
 ('천재', 0.5925087332725525),
 ('영화감독', 0.5785976648330688),
 ('핀처', 0.5773937106132507),
 ('린치', 0.5688480138778687),
 ('능력', 0.5647576451301575),
 ('서극', 0.5586300492286682),
 ('여균동', 0.5543100833892822)]

In [24]:
# 기본은 코사인 유사도 => "cosmul"
model_cbow_model.wv.most_similar_cosmul(positive=['여자', '배우'], negative=['남자'])

[('연기자', 0.90570467710495),
 ('배우다', 0.8848891854286194),
 ('여배우', 0.8822849988937378),
 ('조연', 0.8544042110443115),
 ('시나리오', 0.8283987641334534),
 ('배역', 0.8174970746040344),
 ('대본', 0.8136961460113525),
 ('연출', 0.8056817054748535),
 ('제작비', 0.7905812859535217),
 ('아이돌', 0.7905701398849487)]

In [25]:
model_cbow_model.wv.most_similar_cosmul(positive=['남자', '배우'], negative=['여자'])

[('연기자', 0.9163538813591003),
 ('조연', 0.8647782206535339),
 ('배우진', 0.8587888479232788),
 ('배우다', 0.8470195531845093),
 ('여배우', 0.8294095993041992),
 ('배역', 0.8233481645584106),
 ('엄정화', 0.8203656077384949),
 ('시나리오', 0.818030595779419),
 ('송강호', 0.8157493472099304),
 ('차승원', 0.80646812915802)]

In [26]:
# 그룹간의 유사도 측정
print(model_skipgram_model.wv.n_similarity(['남자', '배우'], ['여자', '감독']))
print(model_skipgram_model.wv.n_similarity(['남자', '배우'], ['남자', '감독']))

0.6814058
0.7202381


In [27]:
# 가장 유사하지 않은 단어를 추출
print(model_skipgram_model.wv.doesnt_match(['영화', '포스터', '감독', '드라마']))

감독


In [28]:
print(model_skipgram_model.wv.doesnt_match(['냉장고', '음식', '밥', '당근']))

음식


기존에 학습된 모델 확인 및 사용

In [29]:
import gensim.downloader as api
from pprint import pprint as pp
pp(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']


In [30]:
glove_vectors_25 = api.load("glove-twitter-25")

In [31]:
glove_vectors_25.most_similar("twitter")

[('facebook', 0.948005199432373),
 ('tweet', 0.9403423070907593),
 ('fb', 0.9342359900474548),
 ('instagram', 0.9104822874069214),
 ('chat', 0.8964964747428894),
 ('hashtag', 0.8885936737060547),
 ('tweets', 0.8878158330917358),
 ('tl', 0.8778460621833801),
 ('link', 0.877821147441864),
 ('internet', 0.8753897547721863)]

In [32]:
glove_vectors_25.most_similar(positive=['woman', 'king'], negative=['man'])

[('meets', 0.8841923475265503),
 ('prince', 0.832163393497467),
 ('queen', 0.8257461190223694),
 ('’s', 0.817409873008728),
 ('crow', 0.813499391078949),
 ('hunter', 0.8131037950515747),
 ('father', 0.8115834593772888),
 ('soldier', 0.81113600730896),
 ('mercy', 0.8082393407821655),
 ('hero', 0.8082263469696045)]

In [33]:
glove_vectors_100 = api.load("glove-wiki-gigaword-100")

In [34]:
glove_vectors_100.most_similar("twitter")

[('facebook', 0.9159134030342102),
 ('myspace', 0.8384657502174377),
 ('youtube', 0.7946597337722778),
 ('blog', 0.7410154938697815),
 ('tweets', 0.726836085319519),
 ('tumblr', 0.7218026518821716),
 ('blogging', 0.7101113200187683),
 ('blogs', 0.6958351731300354),
 ('instagram', 0.6919254660606384),
 ('email', 0.6856087446212769)]

In [35]:
glove_vectors_100.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7698541283607483),
 ('monarch', 0.6843380928039551),
 ('throne', 0.6755736470222473),
 ('daughter', 0.6594556570053101),
 ('princess', 0.6520534157752991),
 ('prince', 0.6517035365104675),
 ('elizabeth', 0.6464517712593079),
 ('mother', 0.6311717629432678),
 ('emperor', 0.6106470823287964),
 ('wife', 0.6098655462265015)]

In [36]:
glove_vectors_100.similar_by_word("cat")

[('dog', 0.8798074126243591),
 ('rabbit', 0.7424427270889282),
 ('cats', 0.732300341129303),
 ('monkey', 0.7288709878921509),
 ('pet', 0.7190139889717102),
 ('dogs', 0.7163873314857483),
 ('mouse', 0.6915250420570374),
 ('puppy', 0.6800068616867065),
 ('rat', 0.6641027331352234),
 ('spider', 0.6501135230064392)]

In [37]:
glove_vectors_100.n_similarity(['sushi', 'shop'],['japanese', 'restaurant'])

0.7066633